In [36]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd
import pycountry
import pypopulation
from geojson import FeatureCollection, dump
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering
from sklearn.neighbors import NearestCentroid
from matplotlib.pyplot import figure

In [25]:
load_dotenv()
TOKEN=os.getenv("MAPBOX_TOKEN")
df = pd.read_csv("C:\MASTER\GIS\Github\Basel_Convention_Scraper/output/processed/clean.csv") # ../output/processed/
origins = list(df.origin.unique())
destinations = list(df.destination.unique())

In [26]:
print(TOKEN)

pk.eyJ1IjoianN0ZXQiLCJhIjoiY2xkN21oejZyMHdjZjNvcW13ZmI1M3E1cSJ9.AHpaXFoc0YOzTXco0RRKBQ


In [27]:
in_first = set(origins)
in_second = set(destinations)
in_second_but_not_in_first = in_second - in_first
countries = origins + list(in_second_but_not_in_first)

In [28]:
df = pd.DataFrame({"country":countries})

In [29]:
headers = {'Accept': 'application/json'}
lst = []
for i in countries:
    obj = {}
    obj["type"]="Feature"
    obj["properties"]={"name": pycountry.countries.get(alpha_2=i).name}
    pop = pypopulation.get_population_a2(i)
    url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{i}.json?&types=country&access_token={TOKEN}"
    r = requests.get(url)
    jason = r.json()
    obj["properties"]["code"] = i.lower()
    coords = jason["features"][0]["center"]
    obj["geometry"] = {"type":"Point", "coordinates": coords}
    lst.append(obj)
    df.loc[df["country"] == i, "lat"] = coords[0]
    df.loc[df["country"] == i, "lon"] = coords[1]
    df.loc[df["country"] == i, "name"] = obj["properties"]["name"]
    df.loc[df["country"] == i, "population"] = pop

In [30]:
with open('C:\MASTER\GIS\Github\Basel_Convention_Scraper/output/processed/countries.geojson', 'w') as f:
    dump(FeatureCollection(lst), f)

In [31]:
df['population'] = df['population'].fillna(0).astype(int)
df

,country,lat,lon,name,population
0,ad,1.576766,42.548654,Andorra,77142
1,ae,10.525771,1.612130,United Arab Emirates,9770529
2,af,66.026471,33.838806,Afghanistan,38041754
3,al,2.632388,28.163240,Albania,2854191
4,am,-97.922211,39.381266,Armenia,2957731
...,...,...,...,...,...
150,as,-170.707833,-14.304405,American Samoa,55312
151,gi,-11.029862,10.785546,Gibraltar,33701
152,sa,44.652426,23.384784,Saudi Arabia,34268528
153,bf,-1.573062,12.277930,Burkina Faso,20321378


In [32]:
df.loc[137, 'population'] = 10457
df.loc[df.index==137] 

,country,lat,lon,name,population
137,bl,28.046788,53.539998,Saint Barthélemy,10457


In [33]:
X = df[["lat","lon"]]

In [34]:
X

,lat,lon
0,1.576766,42.548654
1,10.525771,1.612130
2,66.026471,33.838806
3,2.632388,28.163240
4,-97.922211,39.381266
...,...,...
150,-170.707833,-14.304405
151,-11.029862,10.785546
152,44.652426,23.384784
153,-1.573062,12.277930


In [35]:
hierarchical_cluster = AgglomerativeClustering(n_clusters=15, metric='euclidean', linkage='ward')
labels = hierarchical_cluster.fit_predict(X)
clf = NearestCentroid()
clf.fit(X, labels)
plt.scatter(X["lat"], X.lon, c=labels)
plt.show()

TypeError: __init__() got an unexpected keyword argument 'metric'

In [ ]:
print(len(clf.centroids_))
print(max(labels))
print(len(X))

In [ ]:
lst = []
s = len(X)
# 6 zoom levels
for i in range(6):
    s -= 23
    lst.append(s)
lst

In [ ]:
def func(x,y):
    return y[x-1]

In [ ]:
for count,i in enumerate(lst):
    hierarchical_cluster = AgglomerativeClustering(n_clusters=i, metric='euclidean', linkage='ward')
    labels = hierarchical_cluster.fit_predict(X)
    clf = NearestCentroid()
    clf.fit(X, labels)
    df["temp"] = labels
    df[f"lvl_{count+1}"] = df.apply(lambda x: func(x["temp"], clf.centroids_), axis=1)


In [ ]:
def func(x,y):
    return [x,y]
df[0] = df.apply(lambda x: func(x["lat"], x["lon"]), axis=1)
df.drop(["temp"], inplace=True, axis=1)
df

In [23]:
df.to_csv("C:\MASTER\GIS\Github\Basel_Convention_Scraper/output/processed/coords.csv", index=False)